In [2]:
from trabalhos_transmissao.utils.load_scen import apply_load_scen
from trabalhos_transmissao.utils.wnd_scen import apply_wnd_scen
from power.systems import *
from optimal_power_flow.linear_opf.opf_loss import LinearDispatch

In [5]:
solver = LinearDispatch(IEEE118EOL())

results = solver.solve_loss(verbose=True)

Solução encontrada após 5 iterações.
Custo Total do Sistema: 293755.3754
Perdas Totais do Sistema: 71.1602 MW
Curtailment Total: 0.0000 MW
Shed Total: 319.5482 MW


In [6]:
results["Line"]

,Flow_MW,Losses_MW,Dual_Lower_Cost,Dual_Upper_Cost
Line 1 (1->2),37.024870,0.380373,0.0,0.0
Line 2 (1->3),51.627590,0.314707,0.0,0.0
Line 3 (4->5),-66.602480,0.074450,0.0,0.0
Line 4 (3->5),-3.495405,0.002805,0.0,0.0
Line 5 (5->6),-16.383707,0.030463,0.0,0.0
...,...,...,...,...
Line 182 (114->115),18.968864,0.007890,0.0,0.0
Line 183 (68->116),0.000000,0.000000,0.0,0.0
Line 184 (12->117),20.062748,0.125497,0.0,0.0
Line 185 (75->118),49.610885,0.327150,0.0,0.0
